In [ ]:
import tensorflow as tf
import gradio as gr
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model

ModuleNotFoundError: No module named 'gradio'

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/3

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
pip install tensorflow==2.17.0

In [ ]:
#DenseNet121

import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Layer
from tensorflow.keras.models import Model, load_model
from PIL import Image

# Define any custom layers if needed
class CustomLayer(Layer):
    def __init__(self, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        pass

    def call(self, inputs):
        return tf.keras.backend.relu(inputs)

    def compute_output_shape(self, input_shape):
        return input_shape

# Load the base model without top layers
base_model = tf.keras.applications.DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

# Combine base model and custom top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



# Save the model
model.save('/content/drive/MyDrive/AI/Skin Cancer/densenet121_melanoma_classifier.keras')

# Alternatively, if you saved the entire model including the custom layer
# Load the custom model with custom objects
custom_objects = {'CustomLayer': CustomLayer}


# Ensure the model is already loaded
model_path = '/content/drive/MyDrive/AI/Skin Cancer/densenet121_melanoma_classifier.keras'
model = load_model(model_path, custom_objects=custom_objects)

# Define the class labels
class_labels = ['Not Melanoma', 'Melanoma']

def preprocess_image(img):
    try:
        if isinstance(img, np.ndarray):
            img = Image.fromarray(img)  # Convert numpy array to PIL Image

        img = img.resize((224, 224))  # Resize to match model input size
        img_array = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        return img_array
    except Exception as e:
        return f"Error in preprocessing image: {e}"

def classify_image(img):
    try:
        img_array = preprocess_image(img)
        if isinstance(img_array, str):  # Error message returned as string
            return img_array

        prediction = model.predict(img_array)
        predicted_class_index = np.argmax(prediction)
        predicted_class = class_labels[predicted_class_index]
        return predicted_class
    except Exception as e:
        return f"Error in prediction: {e}"

# Create a Gradio interface
interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(),  # Accepts an image input
    outputs=gr.Label(),  # Outputs the class label
    live=True
)

# Launch the interface
interface.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 738 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3b0f8b2638712317cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3b0f8b2638712317cd.gradio.live


In [ ]:
#efficientnet

# Define any custom layers if needed
class CustomLayer(Layer):
    def __init__(self, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        pass

    def call(self, inputs):
        return tf.keras.backend.relu(inputs)

    def compute_output_shape(self, input_shape):
        return input_shape

# Load the base model without top layers
base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

# Combine base model and custom top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Save the model
model.save('/content/drive/MyDrive/AI/Skin Cancer/efficientnet_melanoma_classifier.keras')

# Load the custom model with custom objects
custom_objects = {'CustomLayer': CustomLayer}

# Ensure the model is already loaded
model_path = '/content/drive/MyDrive/AI/Skin Cancer/efficientnet_melanoma_classifier.keras'
model = load_model(model_path, custom_objects=custom_objects)

# Define the class labels
class_labels = ['Not Melanoma', 'Melanoma']

def preprocess_image(img):
    try:
        if isinstance(img, np.ndarray):
            img = Image.fromarray(img)  # Convert numpy array to PIL Image

        img = img.resize((224, 224))  # Resize to match model input size
        img_array = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        return img_array
    except Exception as e:
        return f"Error in preprocessing image: {e}"

def classify_image(img):
    try:
        img_array = preprocess_image(img)
        if isinstance(img_array, str):  # Error message returned as string
            return img_array

        prediction = model.predict(img_array)
        predicted_class_index = np.argmax(prediction)
        predicted_class = class_labels[predicted_class_index]
        return predicted_class
    except Exception as e:
        return f"Error in prediction: {e}"

# Create a Gradio interface
interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(),  # Accepts an image input
    outputs=gr.Label(),  # Outputs the class label
    live=True
)

# Launch the interface
interface.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 436 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://db97b1b9844ad5e8d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://db97b1b9844ad5e8d3.gradio.live


In [ ]:
#ResNet50

from tensorflow.keras import regularizers
# Define any custom layers if needed
class CustomLayer(Layer):
    def __init__(self, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        pass

    def call(self, inputs):
        return tf.keras.backend.relu(inputs)

    def compute_output_shape(self, input_shape):
        return input_shape

# Load the base model without top layers
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

# Combine base model and custom top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Save the model
model.save('/content/drive/MyDrive/AI/Skin Cancer/resnet50_melanoma_classifier.keras')

# Load the custom model with custom objects
custom_objects = {'CustomLayer': CustomLayer}

# Ensure the model is already loaded
model_path = '/content/drive/MyDrive/AI/Skin Cancer/resnet50_melanoma_classifier.keras'
model = load_model(model_path, custom_objects=custom_objects)

# Define the class labels
class_labels = ['Not Melanoma', 'Melanoma']

def preprocess_image(img):
    try:
        if isinstance(img, np.ndarray):
            img = Image.fromarray(img)  # Convert numpy array to PIL Image

        img = img.resize((224, 224))  # Resize to match model input size
        img_array = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        return img_array
    except Exception as e:
        return f"Error in preprocessing image: {e}"

def classify_image(img):
    try:
        img_array = preprocess_image(img)
        if isinstance(img_array, str):  # Error message returned as string
            return img_array

        prediction = model.predict(img_array)
        predicted_class_index = np.argmax(prediction)
        predicted_class = class_labels[predicted_class_index]
        return predicted_class
    except Exception as e:
        return f"Error in prediction: {e}"

# Create a Gradio interface
interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(),  # Accepts an image input
    outputs=gr.Label(),  # Outputs the class label
    live=True
)

# Launch the interface
interface.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 438 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e031000b3400b44546.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e031000b3400b44546.gradio.live
